In [11]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For oversampling
from collections import Counter
from imblearn.over_sampling import ADASYN
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier

In [12]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\CompleteDatasetApproach2.csv")
#Removing irrelevant columns
df = df.drop(['PlayerName', 'Week'], axis = 1)
features = df.drop(['InjuredNextWeek'], axis = 1)
target = df['InjuredNextWeek']

In [13]:
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(target)
print(imbal_proportion)
#Oversampling the training set
oversample = ADASYN()
featuresi, targeti = oversample.fit_resample(features, target)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(targeti)
print(bal_proportion)

Counter({0: 1158, 1: 29})
Counter({0: 1158, 1: 1152})


In [14]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    FPR = FP / real_values_zero
    FNR = FN / real_values_one
    acc = (TP + TN) / (real_values_one + real_values_zero)
    ba = (sensitivity + specificity) / 2
    f1 = (2*TP) / (2*TP + FP + FN)
    LRplus = sensitivity/(1-specificity)
    LRminus = (1-sensitivity)/specificity
    
    return sensitivity*100, specificity*100, FPR*100, FNR*100, acc*100, ba*100, f1*100, LRplus, LRminus, TP, FP, TN, FN

In [15]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 100691 (43.58917748917749%), FP: 20078 (8.691774891774891%)
TN: 95722 (41.43809523809524%), FN: 14509 (6.280952380952381%)
Sensitivity has mean 87.40538194444443 with variance 3.590237841194057
Specificity has mean 82.66148531951642 with variance 1.401764700618362
False Positive Rate has mean 17.338514680483595 with variance 1.4017647006183607
False Negative Rate has mean 12.594618055555554 with variance 3.5902378411940585
ACC has mean 85.02727272727275 with variance 1.09404452690167
Balanced Accuracy has mean 85.03343363198043 with variance 1.0968772606210182
F1-Score has mean 85.3354259582864 with variance 1.23763459201933
Likelihood ratio positive has mean 5.064799460071707 with variance 0.13394086476741265
Likelihood ratio negative has mean 0.15235012792752442 with variance 0.0005202319064080417


In [16]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 101236 (43.825108225108224%), FP: 20059 (8.683549783549784%)
TN: 95741 (41.44632034632035%), FN: 13964 (6.045021645021645%)
Sensitivity has mean 87.87847222222223 with variance 4.326943479938272
Specificity has mean 82.67789291882556 with variance 1.4245117691451832
False Positive Rate has mean 17.32210708117444 with variance 1.4245117691451816
False Negative Rate has mean 12.121527777777779 with variance 4.326943479938273
ACC has mean 85.27142857142854 with variance 1.2024083131875343
Balanced Accuracy has mean 85.2781825705239 with variance 1.2061664413247881
F1-Score has mean 85.60446149967376 with variance 1.3919355360387926
Likelihood ratio positive has mean 5.096023778902478 with variance 0.12639930104835137
Likelihood ratio negative has mean 0.14657251183487313 with variance 0.0006187176594406421


In [17]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Logistic Regression classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

...

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


TP: 79558 (34.44069264069264%), FP: 57505 (24.893939393939394%)
TN: 58295 (25.235930735930737%), FN: 35642 (15.42943722943723%)
Sensitivity has mean 69.06076388888887 with variance 0.9548882378472234
Specificity has mean 50.34110535405872 with variance 1.2752579487592512
False Positive Rate has mean 49.65889464594128 with variance 1.275257948759252
False Negative Rate has mean 30.939236111111107 with variance 0.9548882378472218
ACC has mean 59.67662337662337 with variance 0.6045784374355797
Balanced Accuracy has mean 59.700934621473806 with variance 0.604158925993395
F1-Score has mean 63.07455658464327 with variance 0.5128638070377357
Likelihood ratio positive has mean 1.391456789308079 with variance 0.001482180898777358
Likelihood ratio negative has mean 0.6149406949066271 with variance 0.000618670735932632


C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 89284 (38.65108225108225%), FP: 66415 (28.751082251082252%)
TN: 49385 (21.37878787878788%), FN: 25916 (11.219047619047618%)
Sensitivity has mean 77.50347222222221 with variance 0.7472993827160495
Specificity has mean 42.646804835924 with variance 0.25151681924346914
False Positive Rate has mean 57.353195164075984 with variance 0.251516819243469
False Negative Rate has mean 22.496527777777775 with variance 0.7472993827160495
ACC has mean 60.029870129870126 with variance 0.3444723674593807
Balanced Accuracy has mean 60.07513852907311 with variance 0.3451151999813396
F1-Score has mean 65.91554394511125 with variance 0.3246563726859958
Likelihood ratio positive has mean 1.351497959211322 with variance 0.0005234437344349811
Likelihood ratio negative has mean 0.5276859054497833 with variance 0.0005610863072919173


In [19]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 72830 (31.528138528138527%), FP: 62615 (27.106060606060606%)
TN: 53185 (23.023809523809526%), FN: 42370 (18.341991341991342%)
Sensitivity has mean 63.22048611111112 with variance 2.011070722415124
Specificity has mean 45.92832469775475 with variance 2.347769962504584
False Positive Rate has mean 54.07167530224525 with variance 2.347769962504585
False Negative Rate has mean 36.779513888888886 with variance 2.0110707224151234
ACC has mean 54.55194805194806 with variance 0.2977970802646125
Balanced Accuracy has mean 54.57440540443294 with variance 0.29734711111670564
F1-Score has mean 58.108120647230145 with variance 0.4734401028575283
Likelihood ratio positive has mean 1.1695996619933229 with variance 0.0005341130472641448
Likelihood ratio negative has mean 0.8009373306822027 with variance 0.0004512898546046895


In [20]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = oversample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 109436 (47.37489177489177%), FP: 9572 (4.143722943722944%)
TN: 106228 (45.986147186147186%), FN: 5764 (2.4952380952380953%)
Sensitivity has mean 94.99652777777777 with variance 0.5032793209876548
Specificity has mean 91.73402417962005 with variance 0.1780927750483979
False Positive Rate has mean 8.265975820379966 with variance 0.17809277504839802
False Negative Rate has mean 5.003472222222223 with variance 0.5032793209876543
ACC has mean 93.36103896103896 with variance 0.1839365828976218
Balanced Accuracy has mean 93.3652759786989 with variance 0.184357848418915
F1-Score has mean 93.45109035338609 with variance 0.1902726338784713
Likelihood ratio positive has mean 11.523239165179472 with variance 0.3715951832307661
Likelihood ratio negative has mean 0.05454774982387848 with variance 6.018275800719563e-05


In [9]:
#Baseline
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []


#All zero vector
b2 = np.ones(len(target))

performance = accuraciz(target, b2)
sensitivities.append(performance[0])
specificities.append(performance[1])
FPRs.append(performance[2])
FNRs.append(performance[3])
ACCs.append(performance[4])
BAs.append(performance[5])
F1s.append(performance[6])
#LRpluses.append(performance[7])
#LRminuses.append(performance[8])
TPs.append(performance[7])
FPs.append(performance[8])
TNs.append(performance[9])
FNs.append(performance[10])
    

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
#print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
#print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 29 (2.443133951137321%), FP: 1158 (97.55686604886267%)
TN: 0 (0.0%), FN: 0 (0.0%)
Sensitivity has mean 100.0 with variance 0.0
Specificity has mean 0.0 with variance 0.0
False Positive Rate has mean 100.0 with variance 0.0
False Negative Rate has mean 0.0 with variance 0.0
ACC has mean 2.443133951137321 with variance 0.0
Balanced Accuracy has mean 50.0 with variance 0.0
F1-Score has mean 4.769736842105264 with variance 0.0
